In [ ]:
# 코랩기준 우선 설치 후 런타인 다시시작 해야됨
!pip install "scikit_learn==0.22.2.post1"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install catboost
!pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import random
import os
import numpy as np

from matplotlib import pyplot as plt

In [ ]:
class CFG:
    SEED = 42

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(CFG.SEED) # Seed 고정

## Data Load

In [ ]:
X_train = pd.read_csv('/content/drive/MyDrive/유전체공모전/data/X_train_new.csv')
X_test = pd.read_csv('/content/drive/MyDrive/유전체공모전/data/X_test_new.csv')
train = pd.read_csv('/content/drive/MyDrive/유전체공모전/data/train.csv')
submit = pd.read_csv('/content/drive/MyDrive/유전체공모전/data/sample_submission.csv')
y_train = train['class']

### Model_Baseline

In [ ]:
from sklearn.feature_selection import SelectPercentile
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, ShuffleSplit, StratifiedKFold

In [ ]:
from sklearn.utils import all_estimators
estimators = all_estimators(type_filter='classifier')
names = []
names_list = []
for name, ClassifierClass in estimators:
    names.append(name)
    names_list.append(name)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import CategoricalNB
from sklearn.multioutput import ClassifierChain
from sklearn.naive_bayes import ComplementNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process.gpc import GaussianProcessClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.semi_supervised import LabelPropagation
from sklearn.semi_supervised import LabelSpreading
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.neural_network.multilayer_perceptron import MLPClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import NearestCentroid
from sklearn.svm import NuSVC
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OutputCodeClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import Perceptron
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import RidgeClassifierCV
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import VotingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

In [ ]:
for i in range(len(names)):
  names[i] = eval(names[i])

In [ ]:
print(names[:5])
print(names_list[:5])

[<class 'sklearn.ensemble._weight_boosting.AdaBoostClassifier'>, <class 'sklearn.ensemble._bagging.BaggingClassifier'>, <class 'sklearn.naive_bayes.BernoulliNB'>, <class 'sklearn.calibration.CalibratedClassifierCV'>, <class 'sklearn.naive_bayes.CategoricalNB'>]
['AdaBoostClassifier', 'BaggingClassifier', 'BernoulliNB', 'CalibratedClassifierCV', 'CategoricalNB']


In [ ]:
"""
Random_state 고정(O):
정상 작동 : 0,1,7,8,9,10,12,13,14,19,22,26,30,31,34,35,37,38
정상 작동,파라미터추가(solver='liblinear') : 20,21

Random_state 고정(X)
random_state 고정X: 2,3,11,15,16,17,18,25,32,36

다른모델 사용(보완모델):
base_estimator(random_state) 오류 : 5
estimator 오류(random_state) : 29
estimator 오류(random_state_X) : 23, 27(다수결 투표),28, 39(stacking), 40(voting)

작동오류:
오류(작동X) : 4(value값에 음수),6(value값에 음수),24(value값에 음수)
dataset오류 : 33(이웃간 거리계산X)
"""

"\nRandom_state 고정(O):\n정상 작동 : 0,1,7,8,9,10,12,13,14,19,22,26,30,31,34,35,37,38\n정상 작동,파라미터추가(solver='liblinear') : 20,21\n\nRandom_state 고정(X)\nrandom_state 고정X: 2,3,11,15,16,17,18,25,32,36\n\n다른모델 사용(보완모델):\nbase_estimator(random_state) 오류 : 5\nestimator 오류(random_state) : 29\nestimator 오류(random_state_X) : 23, 27(다수결 투표),28, 39(stacking), 40(voting)\n\n작동오류:\n오류(작동X) : 4(value값에 음수),6(value값에 음수),24(value값에 음수)\ndataset오류 : 33(이웃간 거리계산X)\n"

In [ ]:
random_state_on = [0,1,7,8,9,10,12,13,14,19,22,26,30,31,34,35,37,38]
random_state_on_lib = [20,21]
random_state_off = [2,3,11,15,16,17,18,25,32,36]

In [ ]:
model_list = []
feature_per = []
cv_scr = []

for i in range(len(random_state_on)):
  # 사용할 모델 설정
  model = names[random_state_on[i]](random_state=CFG.SEED)

  # 각 특성과 타깃(class) 사이에 유의한 통계적 관계가 있는지 계산하여 특성을 선택하는 방법 
  cv_scores = []
  for p in tqdm(range(10,100,1)):
      X_new = SelectPercentile(percentile=p).fit_transform(X_train, y_train)
      skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=CFG.SEED)    
      cv_score = cross_val_score(model, X_new, y_train, scoring='f1_macro', cv=skf).mean()
      cv_scores.append((p,cv_score))

  # Fiting the best percentile
  best_score = cv_scores[np.argmax([score for _, score in cv_scores])]
  print(names_list[random_state_on[i]], best_score)
  feature_per.append(cv_scores[np.argmax([score for _, score in cv_scores])][0])
  cv_scr.append(cv_scores[np.argmax([score for _, score in cv_scores])][1].tolist())
  model_list.append(names_list[random_state_on[i]])
  model.fit(X_train, y_train)
  submit[f'{names_list[random_state_on[i]]}'] = model.predict(X_test)

  0%|          | 0/90 [00:00<?, ?it/s]

AdaBoostClassifier (13, 0.9296245096193946)


  0%|          | 0/90 [00:00<?, ?it/s]

BaggingClassifier (48, 0.9640314637053766)


  0%|          | 0/90 [00:00<?, ?it/s]

...

RidgeClassifier (62, 0.9677825883976778)


  0%|          | 0/90 [00:00<?, ?it/s]

SGDClassifier (66, 0.9640598993220475)


  0%|          | 0/90 [00:00<?, ?it/s]

SVC (61, 0.9641337046017353)


In [ ]:
All_Models_01 = pd.DataFrame(zip(feature_per, cv_scr), index = pd.Index(model_list), columns = ['feature_per', 'cv_scr'])

In [ ]:
model_list = []
feature_per = []
cv_scr = []

for i in range(len(random_state_on_lib)):
  # 사용할 모델 설정
  model = names[random_state_on_lib[i]](solver='liblinear', random_state=CFG.SEED)

  # 각 특성과 타깃(class) 사이에 유의한 통계적 관계가 있는지 계산하여 특성을 선택하는 방법 
  cv_scores = []
  for p in tqdm(range(10,100,1)):
      X_new = SelectPercentile(percentile=p).fit_transform(X_train, y_train)
      skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=CFG.SEED)    
      cv_score = cross_val_score(model, X_new, y_train, scoring='f1_macro', cv=skf).mean()
      cv_scores.append((p,cv_score))

  # Fiting the best percentile
  best_score = cv_scores[np.argmax([score for _, score in cv_scores])]
  print(names_list[random_state_on_lib[i]], best_score)
  feature_per.append(cv_scores[np.argmax([score for _, score in cv_scores])][0])
  cv_scr.append(cv_scores[np.argmax([score for _, score in cv_scores])][1].tolist())
  model_list.append(names_list[random_state_on_lib[i]])
  model.fit(X_train, y_train)
  submit[f'{names_list[random_state_on_lib[i]]}'] = model.predict(X_test)

  0%|          | 0/90 [00:00<?, ?it/s]

LogisticRegression (69, 0.9602362687042995)


  0%|          | 0/90 [00:00<?, ?it/s]

LogisticRegressionCV (51, 0.9606010657430094)


In [ ]:
All_Models_02 = pd.DataFrame(zip(feature_per, cv_scr), index = pd.Index(model_list), columns = ['feature_per', 'cv_scr'])

In [ ]:
model_list = []
feature_per = []
cv_scr = []

for i in range(len(random_state_off)):
  # 사용할 모델 설정
  model = names[random_state_off[i]]()

  # 각 특성과 타깃(class) 사이에 유의한 통계적 관계가 있는지 계산하여 특성을 선택하는 방법 
  cv_scores = []
  for p in tqdm(range(10,100,1)):
      X_new = SelectPercentile(percentile=p).fit_transform(X_train, y_train)
      skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=CFG.SEED)    
      cv_score = cross_val_score(model, X_new, y_train, scoring='f1_macro', cv=skf).mean()
      cv_scores.append((p,cv_score))

  # Fiting the best percentile
  best_score = cv_scores[np.argmax([score for _, score in cv_scores])]
  print(names_list[random_state_off[i]], best_score)
  feature_per.append(cv_scores[np.argmax([score for _, score in cv_scores])][0])
  cv_scr.append(cv_scores[np.argmax([score for _, score in cv_scores])][1].tolist())
  model_list.append(names_list[random_state_off[i]])
  model.fit(X_train, y_train)
  submit[f'{names_list[random_state_off[i]]}'] = model.predict(X_test)

  0%|          | 0/90 [00:00<?, ?it/s]

BernoulliNB (35, 0.8843687806139011)


  0%|          | 0/90 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:946: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:946: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:946: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:946: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:946: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("L

CalibratedClassifierCV (25, 0.9602655196465939)


...

/usr/local/lib/python3.8/dist-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.8/dist-packages/sklearn/discriminant_analysis.py:715: RuntimeWarning: divide by zero encountered in power
  X2 = np.dot(Xm, R * (S ** (-0.5)))
/usr/local/lib/python3.8/dist-packages/sklearn/discriminant_analysis.py:715: RuntimeWarning: invalid value encountered in multiply
  X2 = np.dot(Xm, R * (S ** (-0.5)))
/usr/local/lib/python3.8/dist-packages/sklearn/discriminant_analysis.py:718: RuntimeWarning: divide by zero encountered in log
  u = np.asarray([np.sum(np.log(s)) for s in self.scalings_])
/usr/local/lib/python3.8/dist-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.8/dist-packages/sklearn/discriminant_analysis.py:715: RuntimeWarning: divide by zero encountered in power
  X2 = np.dot(Xm, R * (S ** (-0.5)

QuadraticDiscriminantAnalysis (26, 0.9506532060342803)


/usr/local/lib/python3.8/dist-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.8/dist-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.8/dist-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.8/dist-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.8/dist-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.8/dist-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


  0%|          | 0/90 [00:00<?, ?it/s]

RidgeClassifierCV (62, 0.9677825883976778)


In [ ]:
All_Models_03 = pd.DataFrame(zip(feature_per, cv_scr), index = pd.Index(model_list), columns = ['feature_per', 'cv_scr'])

In [ ]:
names_bst = ['LGBMClassifier','XGBClassifier','CatBoostClassifier']
names_bst_list = ['LGBMClassifier','XGBClassifier','CatBoostClassifier']
for i in range(len(names_bst)):
  names_bst[i] = eval(names_bst[i])
boosting = [0,1,2]

In [ ]:
model_list = []
feature_per = []
cv_scr = []

for i in range(len(boosting)):
  # 사용할 모델 설정
  model = names_bst[boosting[i]](random_state=CFG.SEED)

  # 각 특성과 타깃(class) 사이에 유의한 통계적 관계가 있는지 계산하여 특성을 선택하는 방법 
  cv_scores = []
  for p in tqdm(range(10,100,1)):
      X_new = SelectPercentile(percentile=p).fit_transform(X_train, y_train)
      skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=CFG.SEED)    
      cv_score = cross_val_score(model, X_new, y_train, scoring='f1_macro', cv=skf).mean()
      cv_scores.append((p,cv_score))

  # Fiting the best percentile
  best_score = cv_scores[np.argmax([score for _, score in cv_scores])]
  print(names_bst_list[boosting[i]], best_score)
  feature_per.append(cv_scores[np.argmax([score for _, score in cv_scores])][0])
  cv_scr.append(cv_scores[np.argmax([score for _, score in cv_scores])][1].tolist())
  model_list.append(names_bst_list[boosting[i]])
  model.fit(X_train, y_train)
  submit[f'{names_bst_list[boosting[i]]}'] = model.predict(X_test)

  0%|          | 0/90 [00:00<?, ?it/s]

LGBMClassifier (60, 0.9569999797020257)


  0%|          | 0/90 [00:00<?, ?it/s]

XGBClassifier (74, 0.9564724540875436)


  0%|          | 0/90 [00:00<?, ?it/s]

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
5:	learn: 0.7298522	total: 41.6ms	remaining: 6.88s
6:	learn: 0.6930001	total: 48.4ms	remaining: 6.87s
7:	learn: 0.6612399	total: 55.1ms	remaining: 6.83s
8:	learn: 0.6279716	total: 62ms	remaining: 6.83s
9:	learn: 0.5995672	total: 69.1ms	remaining: 6.84s
10:	learn: 0.5696033	total: 76.5ms	remaining: 6.88s
11:	learn: 0.5395711	total: 79.4ms	remaining: 6.54s
12:	learn: 0.5113470	total: 86.5ms	remaining: 6.57s
13:	learn: 0.4880915	total: 93.7ms	remaining: 6.6s
14:	learn: 0.4633162	total: 101ms	remaining: 6.63s
15:	learn: 0.4443852	total: 108ms	remaining: 6.67s
16:	learn: 0.4253519	total: 116ms	remaining: 6.69s
17:	learn: 0.4093390	total: 123ms	remaining: 6.73s
18:	learn: 0.3921831	total: 130ms	remaining: 6.74s
19:	learn: 0.3761957	total: 137ms	remaining: 6.73s
20:	learn: 0.3609467	total: 145ms	remaining: 6.75s
21:	learn: 0.3479226	total: 152ms	remaining: 6.75s
22:	learn: 0.3336041	total: 159ms	remaining: 6.74s
23:	learn: 0.3215752	total: 166ms	remaining: 

In [ ]:
All_Models_04 = pd.DataFrame(zip(feature_per, cv_scr), index = pd.Index(model_list), columns = ['feature_per', 'cv_scr'])

In [ ]:
all_models_score = pd.concat([All_Models_01,All_Models_02,All_Models_03,All_Models_04]).sort_values(by = 'cv_scr', ascending=False)

In [ ]:
submit = submit.drop(labels='class',axis=1)

In [ ]:
# all_models_score.to_csv('/content/drive/MyDrive/유전체공모전/data/models_score.csv')
# submit.to_csv('/content/drive/MyDrive/유전체공모전/data/all_submit.csv', index=False)

## Emsemble

In [ ]:
all_models_score = all_models_score.reset_index()

In [ ]:
all_models_score

,index,feature_per,cv_scr
0,ExtraTreesClassifier,92,0.975208
1,Perceptron,55,0.971353
2,RidgeClassifierCV,62,0.967783
3,RidgeClassifier,62,0.967783
4,MLPClassifier,53,0.967579
5,SVC,61,0.964134
6,DecisionTreeClassifier,44,0.964134
7,SGDClassifier,66,0.964060
8,BaggingClassifier,48,0.964031
9,LinearSVC,51,0.960716


In [ ]:
submit_01 = submit.drop(labels=['id','DummyClassifier','LinearDiscriminantAnalysis'],axis=1)

In [ ]:
Last_submmit = pd.DataFrame(submit['id'])
Last_submmit['class'] = submit_01.agg(pd.Series.mode,axis=1)

In [ ]:
# submit_01['sum'] = submit_01.sum(axis=1)
# submit_01['A'] = submit_01['sum'].apply(lambda x: x.count('A') )
# submit_01['B'] = submit_01['sum'].apply(lambda x: x.count('B') )
# submit_01['C'] = submit_01['sum'].apply(lambda x: x.count('C') )

In [ ]:
# submit_abc = submit_01.iloc[:,-3:]

In [ ]:
# Last_submmit.to_csv('/content/drive/MyDrive/유전체공모전/submissions/Fine_01.csv', index=False)